# 01. Data Processing and Feature Engineering

This notebook loads the raw data, performs cleaning and feature engineering, and splits the data into Train/Validation/Test sets. These processed datasets are saved to be used by all subsequent model notebooks, ensuring consistency.

In [1]:
# Imports
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

# Settings
pd.set_option('display.max_columns', None)

# Create directories
os.makedirs('../data/processed', exist_ok=True)
print(" Directories checked")

 Directories checked


## 1. Load Data

In [2]:
# Load raw data
# REMOVED try-except to Ensure Fail-Fast behavior
if not os.path.exists('../data/raw/subscriptions.csv'):
    raise FileNotFoundError("subscriptions.csv not found in ../data/raw/")
if not os.path.exists('../data/raw/daily_usage.csv'):
    raise FileNotFoundError("daily_usage.csv not found in ../data/raw/")

subscriptions = pd.read_csv('../data/raw/subscriptions.csv')
daily_usage = pd.read_csv('../data/raw/daily_usage.csv')
print(f"Subscriptions: {subscriptions.shape}")
print(f"Daily usage: {daily_usage.shape}")
    
print("\nSubscriptions Sample:")
display(subscriptions.head(3))

Subscriptions: (503, 19)
Daily usage: (11685, 21)

Subscriptions Sample:


,subscription_id,subscription_created_at,vendor,v2_segment,v2_modules,naf_code,naf_section,revenue_range,employee_count,regional_pole,market,legal_structure,company_age_in_years,company_age_group,subscription_status,trial_starts_at,trial_ends_at,canceled_at,first_paid_invoice_paid_at
0,sub_1RaeVQBY8MfD2ZCIzWApEyq7,2025-06-16T14:59:27,CA,TPE,"[""module_achats_tpe"" ""module_ventes_tpe""]",66.30Z,ACTIVITÉS FINANCIÈRES ET D'ASSURANCE,500k€ à 1M€,de 3 à 9 salariés,Centre-est,PRO,"SASU, société par actions simplifiée uniperson...",0.0,Moins d'un an d'ancienneté,canceled,2025-06-16T14:59:27,2025-07-01T14:59:27,2025-07-22T16:01:13,NaN
1,sub_1ROctUBY8MfD2ZCISlReEeV9,2025-05-14T10:50:36,CA,PME,"[""module_achats_pme"" ""module_ventes_pme""]",Non renseigné,Non renseigné,3M€ à 8M€,+de 10 salariés,Paris et Ile-de-France,PRO,"SARL, société à responsabilité limitée",38.0,Plus de 3 ans d'ancienneté,canceled,2025-05-14T10:50:36,2025-05-29T10:50:36,2025-05-27T08:03:00,NaN
2,sub_1RNbRKBY8MfD2ZCIAiKclYLv,2025-05-11T15:05:17,CA,Independant,NaN,Non renseigné,Non renseigné,0 à 150k€,Indépendant / - 3 salariés,Centre Loire,PRO,Entrepreneur individuel,35.0,Plus de 3 ans d'ancienneté,active,2025-05-11T15:05:17,2025-05-26T15:05:17,NaN,2025-05-26T16:06:48


## 2. Data Cleaning and Feature Engineering

In [3]:
# Convert dates
date_cols = ['trial_starts_at', 'trial_ends_at', 'first_paid_invoice_paid_at']
for col in date_cols:
    subscriptions[col] = pd.to_datetime(subscriptions[col], errors='coerce')

# Calculate trial duration
subscriptions['trial_duration'] = (
    subscriptions['trial_ends_at'] - subscriptions['trial_starts_at']
).dt.days

# Filter to 15-day trials only (as per case study)
subscriptions_15d = subscriptions[subscriptions['trial_duration'] == 15].copy()
print(f"Filtered to 15-day trials: {len(subscriptions_15d)} (from {len(subscriptions)})")

# Define target: converted if they have a paid invoice
subscriptions_15d['converted'] = subscriptions_15d['first_paid_invoice_paid_at'].notna().astype(int)
print(f"Conversion Rate: {subscriptions_15d['converted'].mean():.2%}")

Filtered to 15-day trials: 415 (from 503)
Conversion Rate: 60.72%


In [4]:
# Aggregate usage features per trial
usage_cols = [col for col in daily_usage.columns if col.startswith('nb_')]
print(f"Found {len(usage_cols)} usage columns: {usage_cols[:5]}...")

if not usage_cols:
    raise ValueError("No columns starting with 'nb_' found in daily_usage.csv")

# Aggregate: sum, mean, max, std for each trial
usage_agg = daily_usage.groupby('subscription_id')[usage_cols].agg(
    ['sum', 'mean', 'max', 'std']
).reset_index()

# Flatten column names
usage_agg.columns = ['subscription_id'] + [
    f'{col[0]}_{col[1]}' for col in usage_agg.columns[1:]
]

# Fill NaN std with 0
usage_agg = usage_agg.fillna(0)

# Merge with subscriptions
df = subscriptions_15d.merge(usage_agg, on='subscription_id', how='left')
df = df.fillna(0)

print(f"Final dataset shape: {df.shape}")
display(df.head(3))

Found 19 usage columns: ['nb_transfers_sent', 'nb_transfers_received', 'nb_iban_verification_requests_created', 'nb_mobile_connections', 'nb_banking_accounts_connected']...
Final dataset shape: (415, 97)


,subscription_id,subscription_created_at,vendor,v2_segment,v2_modules,naf_code,naf_section,revenue_range,employee_count,regional_pole,market,legal_structure,company_age_in_years,company_age_group,subscription_status,trial_starts_at,trial_ends_at,canceled_at,first_paid_invoice_paid_at,trial_duration,converted,nb_transfers_sent_sum,nb_transfers_sent_mean,nb_transfers_sent_max,nb_transfers_sent_std,nb_transfers_received_sum,nb_transfers_received_mean,nb_transfers_received_max,nb_transfers_received_std,nb_iban_verification_requests_created_sum,nb_iban_verification_requests_created_mean,nb_iban_verification_requests_created_max,nb_iban_verification_requests_created_std,nb_mobile_connections_sum,nb_mobile_connections_mean,nb_mobile_connections_max,nb_mobile_connections_std,nb_banking_accounts_connected_sum,nb_banking_accounts_connected_mean,nb_banking_accounts_connected_max,nb_banking_accounts_connected_std,nb_products_created_sum,nb_products_created_mean,nb_products_created_max,nb_products_created_std,nb_client_invoices_created_sum,nb_client_invoices_created_mean,nb_client_invoices_created_max,nb_client_invoices_created_std,nb_invoices_created_from_quote_sum,nb_invoices_created_from_quote_mean,nb_invoices_created_from_quote_max,nb_invoices_created_from_quote_std,nb_client_invoices_auto_imported_sum,nb_client_invoices_auto_imported_mean,nb_client_invoices_auto_imported_max,nb_client_invoices_auto_imported_std,nb_customers_created_sum,nb_customers_created_mean,nb_customers_created_max,nb_customers_created_std,nb_client_invoices_sent_sum,nb_client_invoices_sent_mean,nb_client_invoices_sent_max,nb_client_invoices_sent_std,nb_supplier_invoices_auto_imported_sum,nb_supplier_invoices_auto_imported_mean,nb_supplier_invoices_auto_imported_max,nb_supplier_invoices_auto_imported_std,nb_suppliers_created_sum,nb_suppliers_created_mean,nb_suppliers_created_max,nb_suppliers_created_std,nb_supplier_invoices_imported_sum,nb_supplier_invoices_imported_mean,nb_supplier_invoices_imported_max,nb_supplier_invoices_imported_std,nb_transactions_reconciled_sum,nb_transactions_reconciled_mean,nb_transactions_reconciled_max,nb_transactions_reconciled_std,nb_exports_created_sum,nb_exports_created_mean,nb_exports_created_max,nb_exports_created_std,nb_payslips_created_sum,nb_payslips_created_mean,nb_payslips_created_max,nb_payslips_created_std,nb_expense_reports_created_sum,nb_expense_reports_created_mean,nb_expense_reports_created_max,nb_expense_reports_created_std,nb_accountant_users_created_sum,nb_accountant_users_created_mean,nb_accountant_users_created_max,nb_accountant_users_created_std
0,sub_1RaeVQBY8MfD2ZCIzWApEyq7,2025-06-16T14:59:27,CA,TPE,"[""module_achats_tpe"" ""module_ventes_tpe""]",66.30Z,ACTIVITÉS FINANCIÈRES ET D'ASSURANCE,500k€ à 1M€,de 3 à 9 salariés,Centre-est,PRO,"SASU, société par actions simplifiée uniperson...",0.0,Moins d'un an d'ancienneté,canceled,2025-06-16 14:59:27,2025-07-01 14:59:27,2025-07-22T16:01:13,0,15.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000,0,0.0,0,0.0,0,0.0,1,0.0625,1,0.25000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0000,0,0.000000,0,0.0,0,0.0,0,0.0000,0,0.00,0,0.0000,0,0.00,0,0.0000,0,0.00,0,0.0,0,0.0,0,0.0000,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,sub_1ROctUBY8MfD2ZCISlReEeV9,2025-05-14T10:50:36,CA,PME,"[""module_achats_pme"" ""module_ventes_pme""]",Non renseigné,Non renseigné,3M€ à 8M€,+de 10 salariés,Paris et Ile-de-France,PRO,"SARL, société à responsabilité limitée",38.0,Plus de 3 ans d'ancienneté,canceled,2025-05-14 10:50:36,2025-05-29 10:50:36,2025-05-27T08:03:00,0,15.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000,0,0.0,0,0.0,0,0.0,0,0.0000,0,0.00000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0000,0,0.000000,0,0.0,0,0.0,1,0.0625,1,0.25,1,0.0625,1,0.25,1,0.0625,1,0.25,0,0.0,0,0.0,1,0.0625,1,0.25,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,sub_1RNbRKBY8MfD2ZCIAiKclYLv,2025-05-11T15:05:17,CA,Independant,0,Non renseigné,Non renseigné,0 à 150k€,Indépendant / - 3 salariés,Centre Loire,PRO,Entrepreneur individuel,35.0,

## 3. Train / Validation / Test Split

We use a 64% Train / 16% Validation / 20% Test split strategy. Stratified by target variable.

In [5]:
# Select features for modeling
# Use only numerical usage features
feature_cols = [col for col in df.columns if col.startswith('nb_')]
X = df[feature_cols].values
y = df['converted'].values

print(f"Features: {X.shape[1]}")
print(f"Samples: {len(y)}")

# First split: Train+Val (80%) vs Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: Train (80% of 80% = 64%) vs Val (20% of 80% = 16%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

print(f"Train set: {X_train.shape} ({len(X_train)/len(X):.1%})")
print(f"Val set:   {X_val.shape} ({len(X_val)/len(X):.1%})")
print(f"Test set:  {X_test.shape} ({len(X_test)/len(X):.1%})")

Features: 76
Samples: 415
Train set: (265, 76) (63.9%)
Val set:   (67, 76) (16.1%)
Test set:  (83, 76) (20.0%)


## 4. Scaling

Fit StandardScaler on Training data only, then transform Val and Test.

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(" Data scaled")

 Data scaled


## 5. Save Processed Data

Saving as pickle files for efficiency and to preserve numpy arrays exactly.

In [7]:
data_to_save = {
    'X_train': X_train, 'y_train': y_train,
    'X_val': X_val, 'y_val': y_val,
    'X_test': X_test, 'y_test': y_test,
    'X_train_scaled': X_train_scaled,
    'X_val_scaled': X_val_scaled,
    'X_test_scaled': X_test_scaled,
    'scaler': scaler,
    'feature_names': feature_cols
}

with open('../data/processed/churn_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

print(" Processed data saved to ../data/processed/churn_data.pkl")

 Processed data saved to ../data/processed/churn_data.pkl
